# Setting

In [26]:
import pandas as pd
import numpy as np
from copy import deepcopy
import random
import re
import openai
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Bảng dữ liệu

## Loại biểu đồ

In [4]:
keyword = pd.read_excel("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/keyword_from_Mr_Hao.xlsx")
keyword = keyword.apply(lambda col: col.str.lower() if col.dtype!='int64' else col)
keyword

,Tên biểu đồ,Code chart,Keyword
0,biểu đồ cảnh báo 2,alarm_chart_2,thông tin chỉ tiêu
1,biểu đồ rank,rank,xếp hạng
2,biểu đồ đường,line,xu thế
3,biểu đồ cột,column,"tăng trưởng, hoàn thành"
4,biểu đồ cơ cấu,pie,cơ cấu
5,biểu đồ kết hợp đường và cột,combine,tăng giảm
6,biểu đồ thị phần,type_chart_table_thi_phan,thị phần


## CHART 

In [5]:
service_id_all = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/SERVICE_ID_infor.csv")
service_id_list = list(set(service_id_all['SERVICE_ID']))
service_id_dictionary = {u: list(set(service_id_all[service_id_all['SERVICE_ID']==u]['SERVICE_NAME']))[0] for u in service_id_list}
chart_dept = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/D_CHART_DEPT_SERVICE_202307280943.csv")
chart_dept.drop(columns=["GROUP_CHART"], inplace=True)
chart_dept['SERVICE_NAME'] = chart_dept['SERVICE_ID'].apply(lambda i: service_id_dictionary[i] if i in service_id_dictionary.keys() else "")
chart_dept

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,UPDATE_DATE,SERVICE_NAME
0,77254,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,MINI,NGAY,2022-08-08 09:47:22.000,DOANH THU DỊCH VỤ (chi tiết)
1,77255,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,TITLE_MOBILE,NGAY,2022-08-08 09:47:22.000,DOANH THU DỊCH VỤ (chi tiết)
2,77256,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,CHART_RANK_2,NGAY,2022-08-08 09:47:22.000,DOANH THU DỊCH VỤ (chi tiết)
3,77257,S079,970,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,4G Lũy kế
4,77258,S079,971,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,3G Tăng giảm
...,...,...,...,...,...,...,...,...,...
57329,132568,TD,6003,23456,DANH SÁCH NHIỆM VỤ CHẬM TIẾN ĐỘ,TABLE_CONTENT,NaN,2023-05-25 09:06:41.000,Chậm tiến độ
57330,132353,TD,6002,23457,DANH SÁCH SẮP ĐẾN HẠN,TABLE_CONTENT,NaN,2023-05-25 09:12:43.000,Sắp đến hạn
57331,132354,TD,6003,23456,DANH SÁCH NHIỆM VỤ CHẬM TIẾN ĐỘ,TABLE_CONTENT,NaN,2023-05-25 09:06:41.000,Chậm tiến độ
57332,132355,TD,6002,23457,DANH SÁCH SẮP ĐẾN HẠN,TABLE_CONTENT,NaN,2023-05-25 09:12:43.000,Sắp đến hạn


In [6]:
def remove_donvi(title_chart):
    features = title_chart.split("_")
    if " " not in features[0] and features[0] not in ["company", 'info', 'test', 'rank', 'rada', 'table', 'tooltip', '21', 'dgsdf', 'khdn', 'dt-cp-ln', 'arpu', 'mou', 'dou', 'công đoàn']:
        output =  features[0]
        output2 = "_".join(features[1:])
    elif features[0] in ['tập đoàn', 'ban cntt', 'khối tỉnh/tp', 'viettel tỉnh/tp']:
        output = features[0]
        output2 = "_".join(features[1:])
    else:
        output = ""
        output2 = "_".join(features[0:])
    return output, output2
    

def remove_time(title_chart_rm_donvi):
    features = title_chart_rm_donvi.split("_")
    if len(features) == 1:
        features = "_".join(features)
        return features, ""
    output2 = ""
    output = deepcopy(features)
    hehe=0
    for feature in features[::-1]:
        for i in ['ngay', 'thang', 'nam', 'quy', 'ngày', 'tháng', 'năm', 'quý']:
            if i in feature:
                hehe = 1
                output2 = feature
                output.remove(feature)
                break
        if hehe==1:
            break
    
    output = "_".join(output)
    return output, output2

In [7]:
chart_dept_for_title = deepcopy(chart_dept)
chart_dept_for_title.drop(columns=['UPDATE_DATE'], inplace=True)
chart_dept_for_title = chart_dept_for_title.apply(lambda col: col.str.lower() if col.dtype!='int64' else col)
chart_dept_for_title['DONVI_FEATURE'] = chart_dept_for_title['TITLE_CHART'].apply(lambda x: remove_donvi(x)[0])
chart_dept_for_title['TITLE_CHART_RM_DONVI'] = chart_dept_for_title['TITLE_CHART'].apply(lambda x: remove_donvi(x)[1])
chart_dept_for_title['TIME_FEATURE'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[1])
chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[0].replace("_", " "))
chart_dept_for_title.drop(columns=['TITLE_CHART_RM_DONVI'], inplace=True)
chart_dept_for_title[30:50]

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DONVI_FEATURE,TIME_FEATURE,TITLE_CHART_RM_DONVI_TIME
30,77284,t073,276,16365,thuê bao ftth,mini,ngay,fttx - tb thực tăng/giảm,,,thuê bao ftth
31,77285,t073,276,16365,thuê bao ftth,title_mobile,ngay,fttx - tb thực tăng/giảm,,,thuê bao ftth
32,77286,h018,143,12866,hbh_dtdv_từng quý,line,quy,doanh thu dịch vụ (chi tiết),hbh,từng quý,dtdv
33,77287,d511,143,12868,dng_dtdv_từng tháng,line,thang,doanh thu dịch vụ (chi tiết),dng,từng tháng,dtdv
34,77288,d511,143,12870,dng_dtdv_từng năm,line,nam,doanh thu dịch vụ (chi tiết),dng,từng năm,dtdv
35,77289,d511,20214133,12871,dng_dtdv_cơ cấu dịch vụ,share_time,thang,dt dịch vụ nhóm dv của vtt,dng,,dtdv cơ cấu dịch vụ
36,77290,d511,20214132,12871,dng_dtdv_cơ cấu dịch vụ,share_time,thang,dt dịch vụ nhóm dv của vts,dng,,dtdv cơ cấu dịch vụ
37,77291,d511,20212510,12871,dng_dtdv_cơ cấu dịch vụ,share_time,thang,dt dịch vụ nhóm dv của vds,dng,,dtdv cơ cấu dịch vụ
38,77292,d511,20214133,12875,dng_dtdv nhóm vtt_lũy kế ngày,line,ngay,dt dịch vụ nhóm dv của vtt,dng,lũy kế ngày,dtdv nhóm vtt
39,77293,h018,20214132,12615,hbh_dtdv_cơ cấu dịch vụ,share_time,thang,dt dịch vụ nhóm dv của vts,hbh,,dtdv cơ cấu dịch vụ


## Chu kỳ

In [8]:
Chu_ky = list(set(chart_dept_for_title['CYCTIME']))
Chu_ky.remove(np.nan)
Chu_ky

['thang', 'quy', 'nam', 'ngay']

## Chỉ tiêu

In [9]:
Chi_tieu = list(set(chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME']))
len(Chi_tieu)

1427

In [10]:
Chi_tieu[:5]

['tl giao dịch thành công mbccs',
 'xu huong dt dich vu nhom sme vtt',
 'hiệu quả các use-case ứng dụng ai trong lĩnh vực xử lý ngôn ngữ và tiếng nói tự nhiên',
 'xu huong tb thực vtt',
 'thuê bao  viettel money psgd']

## Đơn vị

In [11]:
Don_vi = list(set(chart_dept_for_title['DONVI_FEATURE']))
len(Don_vi)

119

# Prompt instance

## Prepare

In [15]:
keyword

,Tên biểu đồ,Code chart,Keyword
0,biểu đồ cảnh báo 2,alarm_chart_2,thông tin chỉ tiêu
1,biểu đồ rank,rank,xếp hạng
2,biểu đồ đường,line,xu thế
3,biểu đồ cột,column,"tăng trưởng, hoàn thành"
4,biểu đồ cơ cấu,pie,cơ cấu
5,biểu đồ kết hợp đường và cột,combine,tăng giảm
6,biểu đồ thị phần,type_chart_table_thi_phan,thị phần


In [21]:
chart_dept_for_title_prompt = chart_dept_for_title[chart_dept_for_title['TYPE_CHART'].isin(keyword['Code chart'])]
chart_dept_for_title_prompt = chart_dept_for_title_prompt[['CHART_ID', 'TITLE_CHART', 'TYPE_CHART', 'CYCTIME', 'DONVI_FEATURE', 'TIME_FEATURE', 'TITLE_CHART_RM_DONVI_TIME']]
chart_dept_for_title_prompt.rename({'TITLE_CHART_RM_DONVI_TIME':'CHI_TIEU_FEATURE'})
chart_dept_for_title_prompt[-20:]

,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,DONVI_FEATURE,TIME_FEATURE,TITLE_CHART_RM_DONVI_TIME
57166,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động
57167,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động
57168,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động
57169,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động
57170,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động
57171,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động
57172,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động
57173,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động
57174,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động
57175,25647,tập đoàn_td thực dv di động_tiến độ quý các đơ...,column,thang,tập đoàn,tiến độ quý các đơn vị,td thực dv di động


## Prompting